# Designing and Creating a Database

## Introduction
In this project, we will be working with a file of __[Major League Baseball](https://en.wikipedia.org/wiki/Major_League_Baseball)__ games from __[Retrosheet](https://www.retrosheet.org/)__. Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. The main file we will be working from **game_log.csv**, has been produced by combining 127 separate CSV files from retrosheet, and has been pre-cleaned to remove some inconsistencies. The game log has hundreds of data points on each game which we will normalize into several separate tables using SQL, providing a robust database of game-level statistics.

In addition to the main file, we have also included three 'helper' files, also sourced from Retrosheet:

* **park_codes.csv**
* **person_codes.csv**
* **team_codes.csv**

These three helper files in some cases contain extra data, but will also make things easier as they will form the basis for three of our normalized tables.

**game_log_fields.txt** file from Retrosheet which explains the fields included in our main file, which will be useful to assist our EDA

In [65]:
import pandas as pd
import numpy as np
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [66]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


### Getting To Know The Data

In [67]:
gamelog_df = pd.read_csv('game_log.csv', low_memory=False)
gamelog_df.shape

(171907, 161)

In [68]:
gamelog_df.head(5)

date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit protest park_id  attendance  length_minutes v_line_score  \
0      NaN     NaN   FOR01       200.0           120.0    000000000   
1      NaN     NaN   WAS01      5000.0           145.0    107000435   
2      NaN     NaN   RCK01      1000.0           140.0    610020003   
3      NaN     NaN   CHI01      5000.0           150.0    101403111   
4      NaN     NaN   TRO01      3250.0           145.0    000002232   

  h_line_score  v_at_bats  v_hits  v_doubles  v_triples  v_homeruns  v_rbi  \
0    010010000       30.0     4.0        1.0        0.0         0.0    0.0   
1    640113030       41.0    13.0        1.0        2.0         0.0   13.0   
2    010020100       49.0    11.0        1.0        1.0         0.0    8.0   
3    077000000       46.0    15.0        2.0        1.0         2.0   10.0   
4    101003000       46.0    17.0        4.0        1.0         0.0    6.0   

   v_sacrifice_hits  v_sacrifice_flies  v_hit_by_pitch  v_walks  \
0               0.0                0.0             0.0      1.0   
1               0.0                0.0             0.0     18.0   
2               0.0                0.0             0.0      0.0   
3               0.0                0.0             0.0      0.0   
4               0.0                0.0             0.0      2.0   

   v_intentional_walks  v_strikeouts  v_stolen_bases  v_caught_stealing  \
0                  NaN           6.0             1.0                NaN   
1                  NaN           5.0             3.0                NaN   
2                  NaN           1.0             0.0                NaN   
3                  NaN           1.0             0.0                NaN   
4                  NaN           0.0             1.0                NaN   

   v_grounded_into_double  v_first_catcher_interference  v_left_on_base  \
0                    -1.0                           NaN             4.0   
1                    -1.0                           NaN            12.0   
2                    -1.0                           NaN            10.0   
3                    -1.0                           NaN             7.0   
4                    -1.0                           NaN            12.0   

   v_pitchers_used  v_individual_earned_runs  v_team_earned_runs  \
0              1.0                       1.0                 1.0   
1              1.0                       6.0                 6.0   
2              1.0                       0.0                 0.0   
3              1.0                       6.0                 6.0   
4              1.0                       2.0                 2.0   

   v_wild_pitches  v_balks  v_putouts  v_assists  v_errors  v_passed_balls  \
0             0.0      0.0       27.0        9.0       0.0             3.0   
1             1.0      0.0       27.0       13.0      10.0             1.0   
2             2.0      0.0       27.0       12.0       8.0             5.0   
3             0.0      0.0       27.0       15.0      11.0             6.0   
4             0.0      0.0       

In [69]:
gamelog_df.columns

Index(['date', 'number_of_game', 'day_of_week', 'v_name', 'v_league',
       'v_game_number', 'h_name', 'h_league', 'h_game_number', 'v_score',
       ...
       'h_player_7_name', 'h_player_7_def_pos', 'h_player_8_id',
       'h_player_8_name', 'h_player_8_def_pos', 'h_player_9_id',
       'h_player_9_name', 'h_player_9_def_pos', 'additional_info',
       'acquisition_info'],
      dtype='object', length=161)

game_log.csv has data for 171907 games which took place between 1871 and 2016, with the records being chronologically arranged.

For each game we have:

* general information on the game
* team level stats for each team
* a list of players from each team, numbered, with their defensive positions
* the umpires that officiated the game
* some 'awards', like winning and losing pitcher

In [70]:
parkcodes_df = pd.read_csv('park_codes.csv', low_memory=False)
parkcodes_df.shape

(252, 9)

In [71]:
parkcodes_df.head(5)

park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                             NaN  
3  10/03/1993     AL                                             NaN  
4         NaN     AL                                             NaN

In [72]:
parkcodes_df.tail(2)

park_id                                 name  aka       city state  \
250   WOR02  Agricultural County Fair Grounds II  NaN  Worcester    MA   
251   WOR03       Worcester Driving Park Grounds  NaN  Worcester    MA   

          start         end league       notes  
250  08/17/1887  08/17/1887     NL  1 BSN game  
251  10/30/1874  10/30/1874    NaN  1 BS1 game

parl_codes.csv has data regarding the baseball parks which were venues for the games. The prak_id here matchs the park_id in the game log.

In [73]:
personcodes_df = pd.read_csv('person_codes.csv', low_memory=False)
personcodes_df.shape

(20494, 7)

In [74]:
personcodes_df.head(5)

id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN

In [75]:
personcodes_df.tail(2)

id      last first player_debut mgr_debut coach_debut ump_debut
20492  zycht001      Zych  Tony   09/04/2015       NaN         NaN       NaN
20493  thoma102  Thompson   NaN          NaN       NaN         NaN       NaN

person_codes.csv is a list of people with ids. they ids match with those in the game log. There are ids for players, coaches, managers and umpires along with their debut dates.

In [76]:
teamcodes_df = pd.read_csv('team_codes.csv', low_memory=False)
teamcodes_df.shape

(150, 8)

In [77]:
teamcodes_df.head(5)

team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1

In [78]:
teamcodes_df.tail(2)

team_id league  start   end        city   nickname franch_id  seq
148     WSU     UA   1884  1884  Washington  Nationals       WSU    1
149     MIA     NL   2012     0       Miami    Marlins       FLO    2

team_codes.csv has details for individial teams such as their nick names and the city they are from. Each team has a team id and also a franch id. the team id matches with that in the game log.

In [79]:
teamcodes_df['franch_id'].value_counts()

BS1    4
BR3    3
MLA    3
SE1    3
LAA    3
TRN    3
PHA    3
PT1    2
WS2    2
WS9    2
SLU    2
BL2    2
PH1    2
CL3    2
BSP    2
LS2    2
WS1    2
HR1    2
NY2    2
BLA    2
IND    2
FLO    2
CN2    2
CH2    2
SL4    2
MON    2
SL2    2
BR4    1
ALT    1
ML3    1
IN2    1
TBA    1
CHF    1
CNU    1
MID    1
ARI    1
CL2    1
RC2    1
KCF    1
PHU    1
DET    1
WOR    1
CL5    1
TL2    1
MLU    1
CHU    1
BRP    1
PH3    1
CL1    1
CN1    1
WS7    1
CLP    1
HOU    1
WS8    1
CH1    1
IN1    1
BRF    1
BR1    1
WS6    1
ML1    1
SLF    1
CHA    1
SR1    1
WS4    1
TL1    1
PH4    1
WSU    1
BR2    1
BL1    1
NYP    1
BFN    1
NY4    1
SEA    1
BLF    1
PTF    1
PTP    1
PHP    1
BOS    1
TOR    1
SPU    1
CN3    1
FW1    1
SDN    1
CL6    1
KCN    1
BL4    1
SR2    1
WIL    1
KCU    1
RC1    1
WS5    1
WS3    1
PRO    1
KC2    1
KCA    1
RIC    1
BLU    1
KEO    1
CLE    1
BSU    1
BUF    1
LS1    1
CHP    1
TRO    1
PH2    1
DTN    1
COL    1
PHI    1
ELI    1
IN3    1
PTU    1
B

Unlike team_id, franch_id does not appear to be unique for each team. There are some franch_id values that occur for more than one team.

In [80]:
teamcodes_df[teamcodes_df['franch_id'] == 'BS1']

team_id league  start   end       city nickname franch_id  seq
21     BS1    NaN   1871  1875     Boston   Braves       BS1    1
22     BSN     NL   1876  1952     Boston   Braves       BS1    2
23     MLN     NL   1953  1965  Milwaukee   Braves       BS1    3
24     ATL     NL   1966     0    Atlanta   Braves       BS1    4

In [81]:
teamcodes_df[teamcodes_df['franch_id'] == 'IND']

team_id league  start   end          city      nickname franch_id  seq
61     IND     FL   1914  1914  Indianapolis  Hoosier-Feds       IND    1
62     NEW     FL   1915  1915        Newark       Peppers       IND    2

Upon some inspection, it is seen that teams can move between cities and when they do, they get a new team_id, but the franch_id stays the same.

### Importing Data into SQLite

To insert data into a noramalized database, we'll need a single column that can be used as a primary key. The game log file does not have a single column that can be used as a primary key to uniquely identify each game. There are three ways that we could handle this:

* Make a compound primary key, such as a primary key of the date, h_name, and number_of_game columns.
* Insert an integer primary key, eg where the first row is 1, the second row is 2, etc.
* Insert a new column using a custom format.

Because we have not yet normalized our data, it's better not to start with a compound primary key - if we do this, we might end up needing to create a compound key in another table that includes this compound key, which would quickly become cumbersome to work with. An integer primary key is a good choice, but we should first explore whether Retrosheet already have a system for uniquely identifying each game. If they do, this is a better option. It means that if at some later stage we choose to incorporate more detailed game data into our database, the keys we use will be compatible with other sources.

Exploring the Retrosheet site, we can find this __[data dictionary](https://www.retrosheet.org/eventfile.htm)__ for their event files, which list every event within each game. This includes the following description:

*id: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, ATL198304080 should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file.*

This essentially makes a custom key using the three columns. After we import the data, we'll construct this column to use as a primary key in our final database.

#### Creating New Database

In [82]:
db = 'mlb.db'

#### Creating Helper Functions

In [83]:
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = 'SELECT name, type FROM sqlite_master WHERE type IN ("table","view");'
    return run_query(q)

In [84]:
with sqlite3.connect(db) as conn:
    gamelog_df.to_sql('game_log', conn, index = False)
    parkcodes_df.to_sql('park_codes', conn, index = False)
    personcodes_df.to_sql('person_codes', conn, index = False)
    teamcodes_df.to_sql('team_codes', conn, index = False)

In [85]:
show_tables()

name   type
0              person  table
1              league  table
2                team  table
3                game  table
4                park  table
5     appearance_type  table
6   person_appearance  table
7     team_appearance  table
8            game_log  table
9          park_codes  table
10       person_codes  table
11         team_codes  table

In [86]:
c_game_id = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''
try:
    run_command(c_game_id)
except:
    pass

In [87]:
c_add_game_id = '''
UPDATE game_log
SET game_id = date || h_name || number_of_game
WHERE game_id IS NULL
'''
run_command(c_add_game_id)

In [88]:
test = '''
select date, h_name, number_of_game, game_id from game_log
limit 2
'''
run_query(test)

date h_name  number_of_game       game_id
0  18710504    FW1               0  18710504FW10
1  18710505    WS3               0  18710505WS30

### Normalization Opportunities

The following are opportunities for normalization of our data:

* In person_codes, all the debut dates will be able to be reproduced using game log data.
* In team_codes, the start, end and sequence columns will be able to be reproduced using game log data.
* In park_codes, the start and end years will be able to be reproduced using game log data. While technically the state is an attribute of the city, we might not want to have a an incomplete city/state table so we will leave this in.
* There are lots of places in game log where we have a player ID followed by the players name. We will be able to remove this and use the name data in person_codes
* In game_log, all offensive and defensive stats are repeated for the home team and the visiting team. We could break these out and have a table that lists each game twice, one for each team, and cut out this column repetition.
* Similarly, in game_log, we have a listing for 9 players on each team with their positions - we can remove these and have one table that tracks player appearances and their positions.
* We can do a similar thing with the umpires from game_log, instead of listing all four positions as columns, we can put the umpires either in their own table or make one table for players, umpires and managers.
* We have several awards in game_log like winning pitcher and losing pitcher. We can either break these out into their own table, have a table for awards, or combine the awards in with general appearances like the players and umpires.

### Normalized schema

https://raw.githubusercontent.com/dataquestio/solutions/063c14bdf29a00e355493a10762d7b343aa13c25/images/schema-screenshot.png

### Creating Tables Without Foreign Keys

In [89]:
person_table = '''
CREATE TABLE person (
person_id TEXT PRIMARY KEY,
first_name TEXT,
last_name TEXT
)
'''

person_table_data = '''
INSERT OR IGNORE INTO person
SELECT id, first, last FROM person_codes
'''

try:
    run_command(person_table)
except:
    pass

run_command(person_table_data)

test9837 = '''
SELECT * FROM person
LIMIT 2
'''

run_query(test9837)

person_id first_name last_name
0  aardd001      David   Aardsma
1  aaroh101       Hank     Aaron

In [90]:
park_table = '''
CREATE TABLE park (
park_id TEXT PRIMARY KEY,
name TEXT,
nickname TEXT,
city TEXT,
state TEXT,
notes TEXT
)
'''

try:
    run_command(park_table)
except:
    pass

park_table_data = '''
INSERT OR IGNORE INTO park
SELECT park_id, name, aka, city, state, notes FROM park_codes
'''

run_command(park_table_data)

test7542 = '''
SELECT * FROM park
LIMIT 2
'''

run_query(test7542)

park_id            name nickname     city state  \
0   ALB01  Riverside Park     None   Albany    NY   
1   ALT01   Columbia Park     None  Altoona    PA   

                                            notes  
0  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1                                            None

In [91]:
league_table = '''
CREATE TABLE league (
league_id TEXT PRIMARY KEY,
name TEXT
)
'''
try:
    run_command(league_table)
except:
    pass

league_table_data ='''
INSERT OR IGNORE INTO league
VALUES
('NL', 'National League'),
('AL', 'American League'),
('AA', 'American Asscociation'),
('FL', 'Federal League'),
('PL', 'Players League'),
('UA', 'Union Association')
'''

run_command(league_table_data)

test741 = '''
SELECT * FROM league
LIMIT 2
'''

run_query(test741)

league_id             name
0        NL  National League
1        AL  American League

In [92]:
c1 = "DROP TABLE IF EXISTS appearance_type;"

run_command(c1)

app_table = '''
CREATE TABLE appearance_type (
appearance_type_id TEXT PRIMARY KEY,
name TEXT,
category TEXT
)
'''

try:
    run_command(app_table)
except:
    pass

apptype_df = pd.read_csv('appearance_type.csv', low_memory = False)

with sqlite3.connect(db) as conn:
    apptype_df.to_sql('appearance_type', conn, index = False, if_exists = 'append')

test56 = '''
SELECT * FROM appearance_type
LIMIT 2
'''

run_query(test56)

appearance_type_id      name category
0                 O1  Batter 1  offense
1                 O2  Batter 2  offense

### Adding The Team and Game Tables

In [93]:
team_table = '''
CREATE TABLE team (
team_id TEXT PRIMARY KEY,
league_id TEXT,
city TEXT,
nickname TEXT,
franch_id TEXT,
FOREIGN KEY (league_id) REFERENCES league(league_id)
)
'''

try:
    run_command(team_table)
except:
    pass

team_table_data = '''
INSERT OR IGNORE INTO team
SELECT team_id, league, city, nickname, franch_id FROM team_codes
'''

run_command(team_table_data)

test123= '''
SELECT * FROM team
LIMIT 2
'''

run_query(test123)

team_id league_id     city         nickname franch_id
0     ALT        UA  Altoona  Mountain Cities       ALT
1     ARI        NL  Arizona     Diamondbacks       ARI

In [94]:
game_table = '''
CREATE TABLE game (
game_id TEXT PRIMARY KEY,
date TEXT,
number_of_game INTEGER,
park_id TEXT,
length_outs INTEGER,
day BOOLEAN,
completion TEXT,
forefeit TEXT,
protest TEXT,
attendance INTEGER,
length_minutes INTEGER,
additional_info TEXT,
acquisition_info TEXT,
FOREIGN KEY (park_id) REFERENCES park(park_id)
)
'''

try:
    run_command(game_table)
except:
    pass

game_table_data = '''
INSERT OR IGNORE INTO game
SELECT 
game_id,
date,
number_of_game,
park_id,
length_outs,
CASE
    WHEN day_night = 'D' THEN 1
    WHEN day_night = 'N' THEN 0
    ELSE NULL
    END 
    AS day,
completion,
forefeit,
protest,
attendance,
length_minutes,
additional_info,
acquisition_info
FROM game_log
'''

run_command(game_table_data)

test5634= '''
SELECT * FROM game
LIMIT 2
'''

run_query(test5634)

game_id      date  number_of_game park_id  length_outs  day  \
0  18710504FW10  18710504               0   FOR01           54    1   
1  18710505WS30  18710505               0   WAS01           54    1   

  completion forefeit protest  attendance  length_minutes additional_info  \
0       None     None    None         200             120            None   
1       None     None    None        5000             145            HTBF   

  acquisition_info  
0                Y  
1                Y

### Adding the Team Appearance Table

In [95]:
c1 = """
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id)
);
"""

run_command(c1)

c2 = """
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
"""

run_command(c2)

q = """
SELECT * FROM team_appearance
WHERE game_id = (
                 SELECT MIN(game_id) from game
                )
   OR game_id = (
                 SELECT MAX(game_id) from game
                )
ORDER By game_id, home;
"""

run_query(q)

team_id       game_id  home league_id  score line_score  at_bats  hits  \
0     CL1  18710504FW10     0      None      0  000000000       30     4   
1     FW1  18710504FW10     1      None      2  010010000       31     4   
2     MIA  20161002WAS0     0        NL      7  000230020       38    14   
3     WAS  20161002WAS0     1        NL     10  03023002x       30    10   

   doubles  triples  homeruns  rbi  sacrifice_hits  sacrifice_flies  \
0        1        0         0    0               0                0   
1        1        0         0    2               0                0   
2        1        1         2    7               1                0   
3        2        0         1   10               1                1   

   hit_by_pitch  walks  intentional_walks  strikeouts  stolen_bases  \
0             0      1                NaN           6             1   
1             0      1                NaN           0             0   
2             0      3                2.0          10             1   
3             1      8                0.0           3             2   

   caught_stealing  grounded_into_double  first_catcher_interference  \
0              NaN                    -1                         NaN   
1              NaN                    -1                         NaN   
2              1.0                     1                         0.0   
3              0.0                     1                         0.0   

   left_on_base  pitchers_used  individual_earned_runs  team_earned_runs  \
0             4              1                       1                 1   
1             3              1                       0                 0   
2             8              7                      10                10   
3             7              6                       7                 7   

   wild_pitches  balks  putouts  assists  errors  passed_balls  double_plays  \
0             0      0       27        9       0             3             0   
1             0      0       27        3       3             1             1   
2             1      0       24       11       0             0             1   
3             1      0       27       11       0             0             1   

   triple_plays  
0             0  
1             0  
2             0  
3             0

### Adding the Person Appearance Table

In [96]:
c0 = "DROP TABLE IF EXISTS person_appearance"

run_command(c0)

c1 = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c2 = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

run_command(c1)
run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [97]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_person_appearance FROM person_appearance"))

q = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q)

   games_game
0      171907
   games_person_appearance
0                   171907


appearance_id person_id team_id       game_id appearance_type_id  \
0         1646109  porta901    None  20161002WAS0                U1B   
1         1646108  onorb901    None  20161002WAS0                U2B   
2         1646107  kellj901    None  20161002WAS0                U3B   
3         1646110  tumpj901    None  20161002WAS0                UHP   
4         1646111  brica001     MIA  20161002WAS0                ALP   
5         6716279  koeht001     MIA  20161002WAS0                 D1   
6         4744553  telit001     MIA  20161002WAS0                 D2   
7         5589581  bourj002     MIA  20161002WAS0                 D3   
8         4462877  gordd002     MIA  20161002WAS0                 D4   
9         5026229  pradm001     MIA  20161002WAS0                 D5   
10        6434609  hecha001     MIA  20161002WAS0                 D6   
11        5871257  scrux001     MIA  20161002WAS0                 D7   
12        5307905  yelic001     MIA  20161002WAS0                 D8   
13        6152933  hoodd001     MIA  20161002WAS0                 D9   
14        1646113  mattd001     MIA  20161002WAS0                 MM   
15        4462878  gordd002     MIA  20161002WAS0                 O1   
16        4744554  telit001     MIA  20161002WAS0                 O2   
17        5026230  pradm001     MIA  20161002WAS0                 O3   
18        5307906  yelic001     MIA  20161002WAS0                 O4   
19        5589582  bourj002     MIA  20161002WAS0                 O5   
20        5871258  scrux001     MIA  20161002WAS0                 O6   
21        6152934  hoodd001     MIA  20161002WAS0                 O7   
22        6434610  hecha001     MIA  20161002WAS0                 O8   
23        6716280  koeht001     MIA  20161002WAS0                 O9   
24        1646112  koeht001     MIA  20161002WAS0                PSP   
25        1646116  melam001     WAS  20161002WAS0                ASP   
26        1646115  difow001     WAS  20161002WAS0                AWB   
27        1646117  schem001     WAS  20161002WAS0                AWP   
28        4181201  schem001     WAS  20161002WAS0                 D1   
29        3899525  lobaj001     WAS  20161002WAS0                 D2   
30        2772821  zimmr001     WAS  20161002WAS0                 D3   
31        3336173  difow001     WAS  20161002WAS0                 D4   
32        3054497  drews001     WAS  20161002WAS0                 D5   
33        3617849  espid001     WAS  20161002WAS0                 D6   
34        2209469  reveb001     WAS  20161002WAS0                 D7   
35        1927793  turnt001     WAS  20161002WAS0                 D8   
36        2491145  harpb003     WAS  20161002WAS0                 D9   
37        1646114  baked002     WAS  20161002WAS0                 MM   
38        1927794  turnt001     WAS  20161002WAS0                 O1   
39        2209470  reveb001     WAS  20161002WAS0                 O2   
40        2491146  harpb003     WAS  20161002WAS0                 O3   
41        2772822  zimmr001     WAS  20161002WAS0                 O4   
42        3054498  drews001     WAS  20161002WAS0                 O5   
43        3336174  difow001     WAS  20161002WAS0                 O6   
44        3617850  espid001     WAS  20161002WAS0                 O7   
45        3899526  lobaj001     WAS  20161002WAS0                 O8   
46        4181202  schem001     WAS  20161002WAS0                 O9   
47        1646118  schem001     WAS  20161002WAS0                PSP   

                  name category  
0           First Base   umpire  
1          Second Base   umpire  
2           Third Base   umpire  
3           Home Plate   umpire  
4       Losing Pitcher    award  
5              Pitcher  defense  
6              Catcher  defense  
7             1st Base  defense  
8             2nd Base  defense  
9             3rd Base  defense  
10           Shortstop  defense  
11          Left Field  defense  
12        Center Field  defense  

### Removing the Original Tables

In [98]:
show_tables()

name   type
0              person  table
1              league  table
2                team  table
3                game  table
4                park  table
5     team_appearance  table
6            game_log  table
7          park_codes  table
8        person_codes  table
9          team_codes  table
10    appearance_type  table
11  person_appearance  table

In [99]:
tables = [
    "game_log",
    "park_codes",
    "team_codes",
    "person_codes"
]

for t in tables:
    c = '''
    DROP TABLE {}
    '''.format(t)
    
    run_command(c)

show_tables()

name   type
0             person  table
1             league  table
2               team  table
3               game  table
4               park  table
5    team_appearance  table
6    appearance_type  table
7  person_appearance  table